In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path="./env/.env")

api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
# Chat 모델 및 프롬프트 설정
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

# ChatOpenAI 모델 초기화
chat = ChatOpenAI(
    temperature= 0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()] 
)

# 프롬프트 조합 (PipelinePromptTemplate)
# 여러 개의 작은 프롬프트를 조합해서 하나의 큰 프롬프트 완성
# 재사용성, 유지보수성 Good

# 조각 1: 역할 소개 부분
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)
# 필요한 변수: character

# 조각 2: 대화 예시 부분
example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)
# 필요한 변수: example_question, example_answer

# 조각 3: 실제 질문 시작 부분
start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You: 
"""
)
# 필요한 변수: question


# 최종 프롬프트 틀 정의
final = PromptTemplate.from_template(
    """
    {intro}

    {example}

    {start}
"""
)
# 필요한 변수: intro, example, start (위에서 만든 프롬프트 조각들)


# 파이프라인 구성
# (변수명, 프롬프트 객체) 튜플로 매핑
prompts = [
    ("intro", intro),       # final의 {intro}에 intro 프롬프트 결과를 넣음
    ("example", example),   # final의 {example}에 example 프롬프트 결과를 넣음
    ("start", start)        # final의 {start}에 start 프롬프트 결과를 넣음
]

# PipelinePromptTemplate으로 조합
full_prompt = PipelinePromptTemplate(
    final_prompt = final,       # 최종 조합 템플릿
    pipeline_prompts=prompts    # 조각들의 매핑
)

chain = full_prompt | chat
chain.invoke({
    "character": "Pirate", # → intro에서 사용
    "example_question": "What is your location", # → example에서 사용
    "example_answer": "Arrrrg! That is a secret!! Arg arg!!", # → example에서 사용
    "question": "What is your fav food?", # → start에서 사용
})

Arrr matey! Me favorite food be a good ol' plate of salted fish and hardtack! Aye, it be the perfect meal for a pirate like meself! Arrr!

AIMessageChunk(content="Arrr matey! Me favorite food be a good ol' plate of salted fish and hardtack! Aye, it be the perfect meal for a pirate like meself! Arrr!")